The plan is to implement PSO in order to find the different optima from our parameter space. As PSO needs an evaluation function to work, we will use a linear regression model to predict the speed based on the parameters being tried. 

The linear regression is done from the collected data from the robot.

### Imports

In [2]:
# Imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error

import copy
import numpy.random as rnd
import time
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import random
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

### Regression Models

In [3]:
# Get the data
df = pd.read_csv("Final.csv")

In [4]:
df.head()

,Angles_0,Angles_1,freq,amp_1,amp_2,offset_1,offset_2,phase,Potentials_0,Potentials_1,Speed
0,28.55,-5.26,0.9,80.0,48.0,56.0,-34,-52.0,12.0,-27.0,14.622617
1,-100.59,-5.21,0.8,80.0,80.0,-25.0,26,-52.0,-82.0,21.0,12.119482
2,-29.30,4.55,0.8,63.0,79.0,-87.0,-73,22.0,-33.0,8.0,11.847835
3,-46.97,22.27,0.3,48.0,63.0,7.0,-21,11.0,-38.0,10.0,11.480086
4,47.04,-60.64,0.9,80.0,48.0,56.0,-34,55.0,102.0,-42.0,11.266257


In [5]:
df = df.drop(columns = ['offset_1', 'offset_2', 'Potentials_0', 'Potentials_1'], axis = 1)
df.shape

(3107, 7)

In [6]:
# Prepare the data for the model

X = df.drop(columns = 'Speed', axis = 1).values
y = df.Speed.values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 18)

# Create the model
model = LinearRegression()

# Train it
model.fit(X_train, y_train)

# Make prediction
y_pred = model.predict(X_test)
#  Evaluate the model
print('R^2 score: ', model.score(X_test, y_test))
print('MSE : ', mean_squared_error(y_test, y_pred))

R^2 score:  0.4029170781030391
MSE :  2.315759399716146


In [7]:
# Regression Tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

regressor = make_pipeline(StandardScaler(), DecisionTreeRegressor(random_state=0))
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('R^2 score: ',regressor.score(X_test, y_test))
print('MSE : ', mean_squared_error(y_test, y_pred))

R^2 score:  -0.27890325467373356
MSE :  4.960169056467104


In [8]:
# SVR
from sklearn.svm import SVR
import numpy as np

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2, kernel = 'rbf'))
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

print('R^2 score: ',regr.score(X_test, y_test))
print('MSE : ', mean_squared_error(y_test, y_pred))

R^2 score:  0.4164108722422054
MSE :  2.263424322845531


Export the function to be optimized

In [9]:
print(model.intercept_, model.coef_)
intercept = model.intercept_
coefs = model.coef_

0.09409259725585706 [-1.87273739e-03  1.55150949e-03  3.25774771e+00  2.22152347e-02
  2.00946864e-02 -4.76545079e-04]


In [10]:
parameters = [47.04,-60.64,	0.9,	80.0,	48.0,	55.0]

In [11]:
model.predict(np.array(parameters).reshape(1,6))

array([5.55944218])

### PSO

In [12]:
def fitness_function(model, parameters):

  intercept = model.intercept_
  coefs = model.coef_

  sum = 0  
  for i in range(6):
    sum += (coefs[i] * parameters[i])
  f = intercept + sum
  return f

In [13]:
def update_velocity(particle, velocity, pbest, gbest, w_min=0.5, max=1.0, c=0.1):

  # Initialise new velocity array
  num_particle = len(particle)
  new_velocity = np.array([0.0 for i in range(num_particle)])

  # Randomly generate r1, r2 and inertia weight from normal distribution
  r1 = random.uniform(0,max)
  r2 = random.uniform(0,max)
  w = random.uniform(w_min,max)
  c1 = c
  c2 = c
  # Calculate new velocity
  for i in range(num_particle):
    new_velocity[i] = w*velocity[i] + c1*r1*(pbest[i]-particle[i])+c2*r2*(gbest[i]-particle[i])
  return new_velocity

In [14]:
def update_position(particle, velocity):
  # Move particles by adding velocity
  new_particle = particle + velocity
  return new_particle

In [24]:
def pso(population_size, dimension, parameters, generation, fitness_criterion, model):
  # Initialisation
  # Population

  particles = []

  for i in range(population_size):
    # Create a particle and add it to the population
    particle = []
    for j in range(6):
      # Populate the particle with 6 random number from the 6 parameters
      lower_bound = list(parameters.values())[j][0]
      upper_bound = list(parameters.values())[j][1]

      # Some parameters have to be integers
      if j == 0 or j == 1:
        particle.append(random.uniform(lower_bound, upper_bound))
      elif j == 2:
        particle.append(random.randint(lower_bound, upper_bound)/10)
      else:
        particle.append(random.randint(lower_bound, upper_bound))

    particles.append(particle)

  #particles = [[random.uniform(position_min, position_max) for j in range(dimension)] for i in range(population)]
  # Particle's best position
  pbest_position = particles
  # Fitness
  pbest_fitness = []

  for particle in particles:
    fitness = fitness_function(model, particle)
    pbest_fitness.append(fitness)

  # Index of the best particle
  gbest_index = np.argmin(pbest_fitness)
  # Global best particle position
  gbest_position = pbest_position[gbest_index]
  # Velocity (starting from 0 speed)
  velocity = [[0.0 for j in range(dimension)] for i in range(population_size)]
  
  # Loop for the number of generation
  for t in range(generation):
    # Stop if the average fitness value reached a predefined success criterion
    if np.average(pbest_fitness) <= fitness_criterion:
      break
    else:
      for n in range(population_size):
        # Update the velocity of each particle
        velocity[n] = update_velocity(particles[n], velocity[n], pbest_position[n], gbest_position)
        # Move the particles to new position
        particles[n] = update_position(particles[n], velocity[n])
    # Calculate the fitness value
    pbest_fitness = [fitness_function(model, particle) for particle in particles]
    # Find the index of the best particle
    gbest_index = np.argmin(pbest_fitness)
    # Update the position of the best particle
    gbest_position = pbest_position[gbest_index]

  # Print the results
  print('Global Best Position: ', gbest_position)
  print('Best Fitness Value: ', min(pbest_fitness))
  print('Average Particle Best Fitness Value: ', np.average(pbest_fitness))
  print('Number of Generation: ', t)


In [26]:
population_size = 100
dimension = 6
generation = 400
fitness_criterion = 10e-4

In [27]:
parameters = {
    'Angles_0' : [-180, 180],
    'Angles_1' : [-180, 180], 
    'freq' : [0, 10],
    'amp_1' : [-90, 90], 
    'amp_2' : [-90, 90],
    'phase' : [-90, 90]
}

In [29]:
pso(population_size, dimension, parameters, generation, fitness_criterion, model)

Global Best Position:  [-6.77383149e+01  1.11203568e+02  1.00000000e-01 -9.00000000e+01
 -6.80000000e+01 -6.00000000e+00]
Best Fitness Value:  -2.6436936903728436
Average Particle Best Fitness Value:  -0.2634258339805192
Number of Generation:  5


In [23]:
random.randint(0, 10)/10

0.1